# Gold Futures Log Returns Forecasting with Chronos

This notebook implements a log returns-based approach for forecasting gold futures prices using Chronos models.

## Key Advantages of Log Returns Approach:
- **Stationarity**: Log returns are typically stationary, unlike absolute prices
- **Normality**: Better approximation to normal distribution
- **Scale Independence**: Unit-free percentages, generalizable across time periods
- **Financial Interpretation**: Direct relationship to risk metrics and portfolio theory

## Analysis Structure:
1. Data Loading and Log Return Calculation
2. Statistical Analysis (Stationarity, Distribution)
3. Chronos Model Configuration for Returns
4. Model Training and Inference
5. Price Reconstruction and Evaluation
6. Comparison with Absolute Price Approach

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Chronos imports
from chronos import ChronosPipeline, ChronosBoltPipeline
import torch

# Statistical analysis
from scipy import stats
try:
    from statsmodels.tsa.stattools import adfuller, kpss
    STATSMODELS_AVAILABLE = True
except ImportError:
    print("⚠️  statsmodels not found. Installing...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "statsmodels"])
    from statsmodels.tsa.stattools import adfuller, kpss
    STATSMODELS_AVAILABLE = True

# Custom utilities
import sys
sys.path.append('./utils')
from log_return_helpers import (
    calculate_log_returns,
    test_stationarity,
    reconstruct_prices,
    calculate_return_metrics,
    prepare_returns_for_chronos,
    analyze_return_distribution
)

# Set plotting style
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('seaborn')
sns.set_palette("husl")

## 1. Data Loading and Log Return Calculation

In [ ]:
# Load gold futures data
data_path = '../gold_futures_analysis/GCUSD_MAX_FROM_PERPLEXITY.csv'
df = pd.read_csv(data_path)

# Convert date column and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Sort by date
df.sort_index(inplace=True)

print(f"Data shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

In [ ]:
# Calculate log returns for different price series
log_returns = {}
price_columns = ['Open', 'High', 'Low', 'Close']

for col in price_columns:
    log_returns[f'{col}_returns'] = calculate_log_returns(df[col])

# Focus on Close price returns for main analysis
close_returns = log_returns['Close_returns']

print(f"Close returns shape: {close_returns.shape}")
print(f"Returns date range: {close_returns.index.min()} to {close_returns.index.max()}")
print(f"\nBasic statistics:")
print(close_returns.describe())

## 2. Statistical Analysis of Log Returns

In [ ]:
# Test stationarity
stationarity_results = test_stationarity(close_returns)

print("=== STATIONARITY TEST RESULTS ===")
print(f"ADF Test:")
print(f"  Statistic: {stationarity_results['adf_statistic']:.4f}")
print(f"  P-value: {stationarity_results['adf_pvalue']:.4f}")
print(f"  Is Stationary: {stationarity_results['adf_is_stationary']}")

print(f"\nKPSS Test:")
print(f"  Statistic: {stationarity_results['kpss_statistic']:.4f}")
print(f"  P-value: {stationarity_results['kpss_pvalue']:.4f}")
print(f"  Is Stationary: {stationarity_results['kpss_is_stationary']}")

# Compare with price stationarity
price_stationarity = test_stationarity(df['Close'])
print(f"\n=== PRICE STATIONARITY (for comparison) ===")
print(f"ADF P-value: {price_stationarity['adf_pvalue']:.4f}")
print(f"Is Stationary: {price_stationarity['adf_is_stationary']}")

In [ ]:
# Analyze return distribution
distribution_stats = analyze_return_distribution(close_returns)

print("=== RETURN DISTRIBUTION ANALYSIS ===")
print(f"Mean: {distribution_stats['mean']:.6f}")
print(f"Std Dev: {distribution_stats['std']:.6f}")
print(f"Skewness: {distribution_stats['skewness']:.4f}")
print(f"Kurtosis: {distribution_stats['kurtosis']:.4f}")
print(f"\nNormality Test:")
print(f"  Jarque-Bera P-value: {distribution_stats['jarque_bera_pvalue']:.4f}")
print(f"  Is Normal: {distribution_stats['is_normal']}")

print(f"\nPercentiles:")
for pct, value in distribution_stats['percentiles'].items():
    print(f"  {pct}: {value:.6f}")

In [ ]:
# Visualization of returns vs prices
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Price series
axes[0, 0].plot(df.index, df['Close'], linewidth=1)
axes[0, 0].set_title('Gold Futures Close Price')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].grid(True, alpha=0.3)

# Log returns series
axes[0, 1].plot(close_returns.index, close_returns, linewidth=0.8, alpha=0.7)
axes[0, 1].set_title('Log Returns')
axes[0, 1].set_ylabel('Log Return')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

# Return distribution
axes[1, 0].hist(close_returns, bins=50, alpha=0.7, density=True)
axes[1, 0].set_title('Log Returns Distribution')
axes[1, 0].set_xlabel('Log Return')
axes[1, 0].set_ylabel('Density')

# Overlay normal distribution
x = np.linspace(close_returns.min(), close_returns.max(), 100)
normal_dist = stats.norm.pdf(x, close_returns.mean(), close_returns.std())
axes[1, 0].plot(x, normal_dist, 'r-', linewidth=2, label='Normal Distribution')
axes[1, 0].legend()

# Q-Q plot
stats.probplot(close_returns, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot vs Normal Distribution')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary comparison
print("\n=== SUMMARY COMPARISON ===")
print("Price Series:")
print(f"  Coefficient of Variation: {df['Close'].std() / df['Close'].mean():.4f}")
print(f"  Is Stationary: {price_stationarity['adf_is_stationary']}")
print(f"\nLog Returns Series:")
print(f"  Coefficient of Variation: {close_returns.std() / abs(close_returns.mean()):.4f}")
print(f"  Is Stationary: {stationarity_results['adf_is_stationary']}")
print(f"  Closer to Normal: {distribution_stats['is_normal']}")

## 3. Chronos Model Configuration for Returns

In [ ]:
# Configuration based on optimal settings from price analysis
CONFIG = {
    'models': {
        'chronos_bolt_base': 'amazon/chronos-bolt-base',
        'chronos_bolt_small': 'amazon/chronos-bolt-small',
        'chronos_t5_base': 'amazon/chronos-t5-base'
    },
    'context_windows': [63, 126, 252],  # 3M, 6M, 12M trading days
    'prediction_horizons': [1, 3, 7],   # 1D, 3D, 1W
    'num_samples': 100,
    'test_split_date': '2021-01-01',
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print(f"Using device: {CONFIG['device']}")
print(f"Available models: {list(CONFIG['models'].keys())}")
print(f"Test split date: {CONFIG['test_split_date']}")

In [ ]:
# Split data for training/testing
split_date = pd.to_datetime(CONFIG['test_split_date'])

train_returns = close_returns[close_returns.index < split_date]
test_returns = close_returns[close_returns.index >= split_date]

print(f"Training period: {train_returns.index.min()} to {train_returns.index.max()}")
print(f"Testing period: {test_returns.index.min()} to {test_returns.index.max()}")
print(f"Training samples: {len(train_returns)}")
print(f"Testing samples: {len(test_returns)}")

# Corresponding price data for reconstruction
train_prices = df['Close'][df.index < split_date]
test_prices = df['Close'][df.index >= split_date]

print(f"\nLast training price: ${train_prices.iloc[-1]:.2f}")
print(f"First test price: ${test_prices.iloc[0]:.2f}")

## 4. Model Training and Inference

In [ ]:
# Load models
models = {}

for model_name, model_id in CONFIG['models'].items():
    print(f"Loading {model_name}...")
    if 'bolt' in model_name:
        pipeline = ChronosBoltPipeline.from_pretrained(model_id, device_map=CONFIG['device'])
    else:
        pipeline = ChronosPipeline.from_pretrained(model_id, device_map=CONFIG['device'])
    models[model_name] = pipeline
    print(f"  ✓ {model_name} loaded successfully")

print(f"\nLoaded {len(models)} models")

In [ ]:
# Systematic forecasting with different configurations
results = []

for model_name, pipeline in models.items():
    for context_window in CONFIG['context_windows']:
        for horizon in CONFIG['prediction_horizons']:
            
            print(f"\nRunning {model_name} - Context: {context_window} - Horizon: {horizon}")
            
            # Prepare data
            if len(train_returns) < context_window:
                print(f"  ⚠️  Insufficient training data for context window {context_window}")
                continue
            
            # Use training data for context
            context_data = train_returns.iloc[-context_window:].values
            
            # Generate forecasts
            try:
                forecast = pipeline.predict(
                    context=torch.tensor(context_data, dtype=torch.float32),
                    prediction_length=horizon,
                    num_samples=CONFIG['num_samples']
                )
                
                # Extract predictions (mean across samples)
                predictions = forecast[0].median(dim=0).values.numpy()
                
                # Get actual returns for comparison
                actual_returns = test_returns.iloc[:horizon].values
                
                if len(actual_returns) == horizon:
                    # Calculate return metrics
                    return_metrics = calculate_return_metrics(
                        pd.Series(actual_returns), 
                        pd.Series(predictions)
                    )
                    
                    # Reconstruct prices
                    initial_price = train_prices.iloc[-1]
                    
                    # Predicted prices
                    predicted_prices = []
                    current_price = initial_price
                    for ret in predictions:
                        current_price = current_price * np.exp(ret)
                        predicted_prices.append(current_price)
                    
                    # Actual prices
                    actual_prices = test_prices.iloc[:horizon].values
                    
                    # Price-based metrics
                    price_mae = np.mean(np.abs(actual_prices - predicted_prices))
                    price_mape = np.mean(np.abs((actual_prices - predicted_prices) / actual_prices)) * 100
                    
                    # Store results
                    result = {
                        'model': model_name,
                        'context_window': context_window,
                        'horizon': horizon,
                        'return_mae': return_metrics['mae'],
                        'return_rmse': return_metrics['rmse'],
                        'hit_rate': return_metrics['hit_rate'],
                        'volatility_ratio': return_metrics['volatility_ratio'],
                        'price_mae': price_mae,
                        'price_mape': price_mape,
                        'predicted_returns': predictions,
                        'actual_returns': actual_returns,
                        'predicted_prices': predicted_prices,
                        'actual_prices': actual_prices
                    }
                    
                    results.append(result)
                    
                    print(f"  ✓ Return MAE: {return_metrics['mae']:.6f}")
                    print(f"  ✓ Hit Rate: {return_metrics['hit_rate']:.3f}")
                    print(f"  ✓ Price MAE: ${price_mae:.2f}")
                    print(f"  ✓ Price MAPE: {price_mape:.2f}%")
                    
                else:
                    print(f"  ⚠️  Insufficient test data for horizon {horizon}")
                    
            except Exception as e:
                print(f"  ❌ Error: {str(e)}")
                continue

print(f"\nCompleted {len(results)} forecasting experiments")

## 5. Results Analysis and Comparison

In [ ]:
# Convert results to DataFrame for analysis
results_df = pd.DataFrame([
    {k: v for k, v in result.items() if k not in ['predicted_returns', 'actual_returns', 'predicted_prices', 'actual_prices']}
    for result in results
])

print("=== RESULTS SUMMARY ===")
print(results_df.groupby(['model', 'context_window', 'horizon']).agg({
    'return_mae': 'mean',
    'hit_rate': 'mean',
    'price_mae': 'mean',
    'price_mape': 'mean'
}).round(4))

In [ ]:
# Find best configurations
print("\n=== BEST CONFIGURATIONS ===")

# Best by return MAE
best_return_mae = results_df.loc[results_df['return_mae'].idxmin()]
print(f"Best Return MAE: {best_return_mae['return_mae']:.6f}")
print(f"  Model: {best_return_mae['model']}")
print(f"  Context: {best_return_mae['context_window']}")
print(f"  Horizon: {best_return_mae['horizon']}")
print(f"  Hit Rate: {best_return_mae['hit_rate']:.3f}")

# Best by hit rate
best_hit_rate = results_df.loc[results_df['hit_rate'].idxmax()]
print(f"\nBest Hit Rate: {best_hit_rate['hit_rate']:.3f}")
print(f"  Model: {best_hit_rate['model']}")
print(f"  Context: {best_hit_rate['context_window']}")
print(f"  Horizon: {best_hit_rate['horizon']}")
print(f"  Return MAE: {best_hit_rate['return_mae']:.6f}")

# Best by price MAE
best_price_mae = results_df.loc[results_df['price_mae'].idxmin()]
print(f"\nBest Price MAE: ${best_price_mae['price_mae']:.2f}")
print(f"  Model: {best_price_mae['model']}")
print(f"  Context: {best_price_mae['context_window']}")
print(f"  Horizon: {best_price_mae['horizon']}")
print(f"  Hit Rate: {best_price_mae['hit_rate']:.3f}")

In [ ]:
# Visualization of results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Return MAE by model
sns.boxplot(data=results_df, x='model', y='return_mae', ax=axes[0, 0])
axes[0, 0].set_title('Return MAE by Model')
axes[0, 0].tick_params(axis='x', rotation=45)

# Hit Rate by model
sns.boxplot(data=results_df, x='model', y='hit_rate', ax=axes[0, 1])
axes[0, 1].set_title('Hit Rate by Model')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random')
axes[0, 1].legend()

# Price MAE by context window
sns.boxplot(data=results_df, x='context_window', y='price_mae', ax=axes[1, 0])
axes[1, 0].set_title('Price MAE by Context Window')

# Price MAPE by horizon
sns.boxplot(data=results_df, x='horizon', y='price_mape', ax=axes[1, 1])
axes[1, 1].set_title('Price MAPE by Horizon')

plt.tight_layout()
plt.show()

## 6. Detailed Analysis of Best Configuration

In [ ]:
# Analyze the best configuration in detail
best_idx = results_df['return_mae'].idxmin()
best_result = results[best_idx]

print(f"=== DETAILED ANALYSIS OF BEST CONFIGURATION ===")
print(f"Model: {best_result['model']}")
print(f"Context Window: {best_result['context_window']} days")
print(f"Prediction Horizon: {best_result['horizon']} days")
print(f"\nReturn Metrics:")
print(f"  MAE: {best_result['return_mae']:.6f}")
print(f"  RMSE: {best_result['return_rmse']:.6f}")
print(f"  Hit Rate: {best_result['hit_rate']:.3f}")
print(f"  Volatility Ratio: {best_result['volatility_ratio']:.3f}")
print(f"\nPrice Metrics:")
print(f"  MAE: ${best_result['price_mae']:.2f}")
print(f"  MAPE: {best_result['price_mape']:.2f}%")

In [ ]:
# Visualization of best result
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Return comparison
dates = test_returns.index[:len(best_result['actual_returns'])]
axes[0, 0].plot(dates, best_result['actual_returns'], 'b-', label='Actual Returns', linewidth=2)
axes[0, 0].plot(dates, best_result['predicted_returns'], 'r--', label='Predicted Returns', linewidth=2)
axes[0, 0].set_title('Return Comparison')
axes[0, 0].set_ylabel('Log Return')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Price comparison
axes[0, 1].plot(dates, best_result['actual_prices'], 'b-', label='Actual Prices', linewidth=2)
axes[0, 1].plot(dates, best_result['predicted_prices'], 'r--', label='Predicted Prices', linewidth=2)
axes[0, 1].set_title('Price Comparison')
axes[0, 1].set_ylabel('Price ($)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Return scatter plot
axes[1, 0].scatter(best_result['actual_returns'], best_result['predicted_returns'], alpha=0.7)
axes[1, 0].plot([-0.1, 0.1], [-0.1, 0.1], 'r--', alpha=0.5)
axes[1, 0].set_xlabel('Actual Returns')
axes[1, 0].set_ylabel('Predicted Returns')
axes[1, 0].set_title('Return Scatter Plot')
axes[1, 0].grid(True, alpha=0.3)

# Error analysis
return_errors = best_result['actual_returns'] - best_result['predicted_returns']
axes[1, 1].hist(return_errors, bins=20, alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Return Prediction Error')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Error Distribution')
axes[1, 1].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Error statistics
print(f"\n=== ERROR ANALYSIS ===")
print(f"Return Error Mean: {return_errors.mean():.6f}")
print(f"Return Error Std: {return_errors.std():.6f}")
print(f"Return Error Skewness: {stats.skew(return_errors):.4f}")
print(f"Return Error Kurtosis: {stats.kurtosis(return_errors):.4f}")

## 7. Comparison with Absolute Price Approach

In [ ]:
# Load results from absolute price approach for comparison
price_results_path = '../gold_futures_analysis/phase1_final_comparison_results.csv'

try:
    price_results = pd.read_csv(price_results_path)
    
    # Find best price-based result
    best_price_result = price_results.loc[price_results['mae'].idxmin()]
    
    print("=== COMPARISON: LOG RETURNS vs ABSOLUTE PRICES ===")
    print(f"\nBest Log Returns Approach:")
    print(f"  Model: {best_result['model']}")
    print(f"  Context: {best_result['context_window']} days")
    print(f"  Horizon: {best_result['horizon']} days")
    print(f"  Price MAE: ${best_result['price_mae']:.2f}")
    print(f"  Price MAPE: {best_result['price_mape']:.2f}%")
    print(f"  Hit Rate: {best_result['hit_rate']:.3f}")
    
    print(f"\nBest Absolute Price Approach:")
    print(f"  Model: {best_price_result['model']}")
    print(f"  Context: {best_price_result['context_window']} days")
    print(f"  Horizon: {best_price_result['horizon']} days")
    print(f"  Price MAE: ${best_price_result['mae']:.2f}")
    print(f"  Price MAPE: {best_price_result['mape']:.2f}%")
    print(f"  Hit Rate: {best_price_result['directional_accuracy']:.3f}")
    
    # Calculate improvement
    mae_improvement = (best_price_result['mae'] - best_result['price_mae']) / best_price_result['mae'] * 100
    mape_improvement = (best_price_result['mape'] - best_result['price_mape']) / best_price_result['mape'] * 100
    hit_rate_improvement = (best_result['hit_rate'] - best_price_result['directional_accuracy']) / best_price_result['directional_accuracy'] * 100
    
    print(f"\n=== IMPROVEMENT ANALYSIS ===")
    print(f"MAE Improvement: {mae_improvement:+.2f}%")
    print(f"MAPE Improvement: {mape_improvement:+.2f}%")
    print(f"Hit Rate Improvement: {hit_rate_improvement:+.2f}%")
    
    if mae_improvement > 0:
        print(f"\n✅ Log returns approach shows {mae_improvement:.1f}% better MAE performance")
    else:
        print(f"\n❌ Log returns approach shows {abs(mae_improvement):.1f}% worse MAE performance")
        
except FileNotFoundError:
    print("❌ Price-based results not found. Run the absolute price analysis first for comparison.")
except Exception as e:
    print(f"❌ Error loading price results: {e}")

## 8. Save Results

In [ ]:
# Save results
results_df.to_csv('./results/log_returns_analysis_results.csv', index=False)

# Save detailed best result
best_result_detailed = {
    'configuration': {
        'model': best_result['model'],
        'context_window': best_result['context_window'],
        'horizon': best_result['horizon']
    },
    'return_metrics': {
        'mae': best_result['return_mae'],
        'rmse': best_result['return_rmse'],
        'hit_rate': best_result['hit_rate'],
        'volatility_ratio': best_result['volatility_ratio']
    },
    'price_metrics': {
        'mae': best_result['price_mae'],
        'mape': best_result['price_mape']
    }
}

import json
with open('./results/best_log_returns_config.json', 'w') as f:
    json.dump(best_result_detailed, f, indent=2)

print("✅ Results saved successfully!")
print(f"  - Summary: ./results/log_returns_analysis_results.csv")
print(f"  - Best config: ./results/best_log_returns_config.json")

## 9. Key Findings and Recommendations

### Statistical Properties
- **Stationarity**: Log returns show [stationarity status] vs prices
- **Distribution**: Returns are [more/less] normally distributed than price changes
- **Volatility**: Returns provide direct volatility modeling capability

### Model Performance
- **Best Configuration**: [model] with [context] context and [horizon] horizon
- **Return Accuracy**: MAE of [value] with [hit_rate] directional accuracy
- **Price Reconstruction**: MAE of $[value] with [mape]% MAPE

### Comparison with Absolute Price Approach
- **Improvement**: [+/-X%] in MAE, [+/-X%] in MAPE
- **Directional Accuracy**: [+/-X%] improvement in hit rate
- **Model Efficiency**: Returns approach shows [better/worse] generalization

### Recommendations
1. **Production Use**: [Recommend/Don't recommend] log returns for gold futures
2. **Optimal Settings**: Use [best_config] for live trading
3. **Risk Management**: Returns provide direct volatility estimates
4. **Ensemble Approach**: Consider combining with absolute price models

### Next Steps
1. **Extended Testing**: Validate on 2022-2024 data
2. **Volatility Modeling**: Implement GARCH-style conditional volatility
3. **Multi-horizon**: Develop path-dependent return forecasting
4. **Feature Engineering**: Add technical indicators to return series